<a href="https://colab.research.google.com/github/amannin2/Process-Controls/blob/main/Copy_of_05_03_Homework_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--NOTEBOOK_HEADER-->
*This notebook contains material from [cbe30338-2021](https://jckantor.github.io/cbe30338-2021);
content is available [on Github](https://github.com/jckantor/cbe30338-2021.git).*


<!--NAVIGATION-->
< [5.2 Linear Blending Problems](https://jckantor.github.io/cbe30338-2021/05.02-Linear-Blending-Problem.html) | [Contents](toc.html) | [Tag Index](tag_index.html) | [5.4 Gasoline Blending](https://jckantor.github.io/cbe30338-2021/05.04-Gasoline-Blending.html) ><p><a href="https://colab.research.google.com/github/jckantor/cbe30338-2021/blob/master/docs/05.03-Homework_5.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://jckantor.github.io/cbe30338-2021/05.03-Homework_5.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>

# 5.3 Homework Assignment 4

Homework assigment 4 is intended to develop your skills in creating linear programming models. The lecture notes (notebook 5.2) will provide with a starting point for the following exercises. The problem data has been changed to provide a more meaningful example for sensitivity analysis.


You have been placed in charge of a metals reuse operation. A collection of raw materials are available with key parameters given in the following table.

| material | % carbon (C) | % copper (Cu) | % manganese (Mn) | amount (kg) | cost (\$/kg) | type |
| :---: | :---: | :---: | :---: | :---: | :---: | :---
| A | 2.5 | 0.0 | 1.3 | 4000 | 1.20 | iron alloy
| B | 3.0 | 0.0 | 0.8 | 3000 | 1.50 | iron alloy
| C | 0.0 | 0.3 | 0.0 | 6000 | 0.90 | iron alloy
| D | 0.0 | 90.0 | 0.0 | 5000 | 1.30 | copper alloy
| E | 0.0 | 96.0 | 4.0 | 2000 | 1.45 | copper alloy
| F | 0.0 | 0.4 | 1.2 | 3000 | 1.20 | aluminum alloy
| G | 0.0 | 0.6 | 0.0 | 2500 | 1.00 | aluminum alloy

A customer has requested 5000 kg of mix satisfying these  specifications:

| Component | min % | max % |
| :---: | :---: | :---: |
| C | 2.0 | 3.0 |
| Cu | 0.4 | 0.6 |
| Mn | 1.2 | 1.65 |

For convenience, the raw material data has been organized as a nested dictionaries.

In [1]:
data = {
    "A": {"C": 2.5, "Cu": 0.0, "Mn": 1.3, "amount": 4000, "cost": 1.20},
    "B": {"C": 3.0, "Cu": 0.0, "Mn": 0.8, "amount": 3000, "cost": 1.50},
    "C": {"C": 0.0, "Cu": 0.3, "Mn": 0.0, "amount": 6000, "cost": 0.90},
    "D": {"C": 0.0, "Cu": 90.0, "Mn": 0.0, "amount": 5000, "cost": 1.30},
    "E": {"C": 0.0, "Cu": 96.0, "Mn": 4.0, "amount": 2000, "cost": 1.45},
    "F": {"C": 0.0, "Cu": 0.4, "Mn": 1.2, "amount": 3000, "cost": 1.20},
    "G": {"C": 0.0, "Cu": 0.6, "Mn": 0.0, "amount": 2500, "cost": 1.00},
}

bound_array = {
    "Ca": {"min": 2.0, "max": 3.0},
    "Cu": {"min": 0.4, "max": 0.6},
    "Mn": {"min": 1.2, "max": 1.65}
}

## 5.3.1 Exercise 1. Getting Started




Considering just the constraint on carbon content, adapt the `brew_blend` function from notebook 5.2 to find a minimum cost blend raw materials that yield 5000 kg of product with a carbon content between 2.0 and 3.0%. For this first exercise you can ignore the bounds on the available amount of each raw material.

* change the name of the function to `metal_blend`.
* `metal_blend` should accept arguments including
    * `data`, 
    * the required mass of final product, and
    *  the acceptable range of carbon content specified as a pair of numbers of  `[lower_bound, upper_bound]`.
* `metal_blend` should return the minimum cost, and the amounts of each raw material to include in the blend.

Demonstrate use of `metal_blend` to compute the required blend. From the results of the calculation to report

* the cost of the blend
* the total mass of the blend
* the mass of each raw material used in the blend
* the composition of the blend with regard to the species carbon, copper and managanese.

You may find it convenient to write a function for this purpose that can be used in the following exercise.



### 5.3.1.1 Solution

In [2]:
import numpy as np
import cvxpy as cp

def metal_blend(data, mass, bound_array):
    
    # Unpack bounds
    lower_bound, upper_bound = bounds

    # create set of components
    components = set(data.keys())
    
    # create variables
    x = {c: cp.Variable(nonneg=True, name=c) for c in components}
    
    # create objective function
    total_cost = sum(x[c]*data[c]['cost'] for c in components)
    
    # create list of constraints
    constraints = [
        0 <= sum(x[c]*data[c]['C']/100 for c in components)/mass*100 - lower_bound,
        0 >= sum(x[c]*data[c]['C']/100 for c in components)/mass*100 - upper_bound,
        mass == sum(x[c] for c in components)
    ]  
    
    # create and solve problem
    problem = cp.Problem(cp.Minimize(total_cost), constraints)
    problem.solve()
    
    # return results
    min_cost = problem.value
    optimal_blend = {c: x[c].value for c in components}
    C_conc = sum(x[c].value*data[c]['C']/100 for c in components)/mass
    Cu_conc = sum(x[c].value*data[c]['Cu']/100 for c in components)/mass
    Mn_conc = sum(x[c].value*data[c]['Mn']/100 for c in components)/mass
    blend_comp = [C_conc, Cu_conc, Mn_conc]
    return min_cost, optimal_blend, blend_comp

In [4]:
bounds = [2,3]
mass = 5000
min_cost, optimal_blend, blend_comp = metal_blend(data, mass, bounds)
print(f'Minimum Cost = {min_cost} dollars')
print(f'Optimal Blend: {optimal_blend}')
print(f'% C: {100*blend_comp[0]}')
print(f'% Cu: {100*blend_comp[1]}')
print(f'% Mn: {100*blend_comp[2]}')

Minimum Cost = 5700.0000002861625 dollars
Optimal Blend: {'E': 1.0271813721011657e-07, 'D': 1.5132256949499066e-07, 'C': 999.999999134046, 'A': 3999.99999950496, 'G': 5.688986266333344e-07, 'B': 3.3114738589179227e-07, 'F': 2.069068901420178e-07}
% C: 1.9999999999511682
% Cu: 0.06000000472885763
% Mn: 1.0400000000561056


## 5.3.2 Exercise 2. Incorporating Constraints

We'll continue this problem by incorporating all of the constraints. The constraiints include:

* Lower and upper bounds on the copper and manganese composition
* Limits on the amount of available raw material

The easiest way to proceed is to copy and paste `metal_blend` into a cell below, then add constraints one at a time until all have been incorporated.

What is the minimum price you would need to charge in $/kg to produce 5,000 kg of requested product?

In [5]:
import numpy as np
import cvxpy as cp

def metal_blend(data, mass, bounds):
    
    # Unpack bounds
    conc = bounds

    # create set of components
    components = set(data.keys())
    
    # create variables
    x = {c: cp.Variable(nonneg=True, name=c) for c in components}
    
    # create objective function
    total_cost = sum(x[c]*data[c]['cost'] for c in components)
    
    # create list of constraints
    constraints = [
        0 <= sum(x[c]*data[c]['C']/100 for c in components)/mass*100 - conc['Ca']['min'],
        0 >= sum(x[c]*data[c]['C']/100 for c in components)/mass*100 - conc['Ca']['max'],
        0 <= sum(x[c]*data[c]['Cu']/100 for c in components)/mass*100 - conc['Cu']['min'],
        0 >= sum(x[c]*data[c]['Cu']/100 for c in components)/mass*100 - conc['Cu']['max'],
        0 <= sum(x[c]*data[c]['Mn']/100 for c in components)/mass*100 - conc['Mn']['min'],
        0 >= sum(x[c]*data[c]['Mn']/100 for c in components)/mass*100 - conc['Mn']['max'], 
        0 >= x['A'] - data['A']['amount'],
        0 >= x['B'] - data['B']['amount'],
        0 >= x['C'] - data['C']['amount'],
        0 >= x['D'] - data['D']['amount'],
        0 >= x['E'] - data['E']['amount'],
        0 >= x['F'] - data['F']['amount'],
        0 >= x['G'] - data['G']['amount'],
        mass == sum(x[c] for c in components)
    ]  
    
    # create and solve problem
    problem = cp.Problem(cp.Minimize(total_cost), constraints)
    problem.solve()
    
    # return results
    min_cost = problem.value
    optimal_blend = {c: x[c].value for c in components}
    C_conc = sum(x[c].value*data[c]['C']/100 for c in components)/mass
    Cu_conc = sum(x[c].value*data[c]['Cu']/100 for c in components)/mass
    Mn_conc = sum(x[c].value*data[c]['Mn']/100 for c in components)/mass
    blend_comp = [C_conc, Cu_conc, Mn_conc]
    return min_cost, optimal_blend, blend_comp

In [6]:
mass = 5000
min_cost, optimal_blend, blend_comp = metal_blend(data, mass, bound_array)
print(f'Minimum Cost = {min_cost} dollars')
print(f'Optimal Blend: {optimal_blend}')
print(f'% C: {100*blend_comp[0]}')
print(f'% Cu: {100*blend_comp[1]}')
print(f'% Mn: {100*blend_comp[2]}')

Minimum Cost = 5887.574276138116 dollars
Optimal Blend: {'E': 27.612721476952302, 'D': 1.6988942590043377e-07, 'C': 397.7630139073558, 'A': 3999.9999994246773, 'G': 2.341794167100359e-06, 'B': 1.3081809082283902e-06, 'F': 574.6242613711506}
% C: 2.0000000004972467
% Cu: 0.5999999774406426
% Mn: 1.1999999999703632


## 5.3.3 Exercise 3. Sensitivity Analysis

One of the important reasons to create optimization models is to understand the value of the raw materials that are available to you, and how to adjust operations to accomodate changing requirements. The is commonly referred to as **sensitivity analysis**.

Using the functions you've created in above exercises, answer the following questions:

1. The customer is very pleased with your initial pricing for 5,000 kg of the desired product, and now wishes to increase the order to 6,000 kg. Does your unit cost ($/kg) go up? If so, explain why your unit cost has gone up.

2. Is there an upper limit on how much product your can provide to this customer? (Use trial and error. To the nearest 100 kg, find the maximum amount of product you can ship to the customer.) What is the unit cost now?

3. After some negotiation, you and your customer settle on an order of 6,500 kg. Now you wish to negotiate with your suppliers for more raw material. How much money to you save for 1 additional kg of raw material "A"? Based on this, what price would you be willing to pay your supplier for additional "A"?



Part 1: Expand order to 6,000 units

In [7]:
mass = 6000
min_cost, optimal_blend, blend_comp = metal_blend(data, mass, bound_array)
print(f'Minimum Cost = {min_cost} dollars')
print(f'Optimal Blend: {optimal_blend}')
print(f'% C: {100*blend_comp[0]}')
print(f'% Cu: {100*blend_comp[1]}')
print(f'% Mn: {100*blend_comp[2]}')

print(f'price per unit = {min_cost/mass} dollars')

Minimum Cost = 7352.1437762038095 dollars
Optimal Blend: {'E': 32.273092308560294, 'D': 8.476167894737313e-07, 'C': 186.41499178198316, 'A': 3999.999997040055, 'G': 1.4098485562594284e-06, 'B': 666.6666695282394, 'F': 1114.6452470836973}
% C: 2.0000000001974763
% Cu: 0.5999999225202137
% Mn: 1.19999999958489
price per unit = 1.2253572960339683 dollars


The cost per unit does increase because there are limits on the amount of the cheapest materials available. This means that to make 6000 kg the more expensive materials have to be used, and these materials increase the unit price.

Part 2: What is the upper limit on how much product can be provided to the customer? Use trial and error. 

In [8]:
mass = 6800
min_cost, optimal_blend, blend_comp = metal_blend(data, mass, bound_array)
print(f'Minimum Cost = {min_cost} dollars')
print(f'Optimal Blend: {optimal_blend}')
print(f'% C: {100*blend_comp[0]}')
print(f'% Cu: {100*blend_comp[1]}')
print(f'% Mn: {100*blend_comp[2]}')

print(f'price per unit = {min_cost/mass} dollars')

Minimum Cost = 8523.799373967979 dollars
Optimal Blend: {'E': 36.00139613232746, 'D': 9.024475587930024e-07, 'C': 17.336585229855814, 'A': 3999.9999968392376, 'G': 3.9791029970022975e-06, 'B': 1200.0000040314483, 'F': 1546.6620128855768}
% C: 2.000000000616535
% Cu: 0.5999999842697598
% Mn: 1.1999999998688484
price per unit = 1.2534999079364675 dollars


The maximum amount of product is 6,800 kg with a unit cost = 1.25 dollars/unit

Part 3: Determine how much money you are willing to spend on an additional unit of A for a sale of 6500 kg and a negotiation to buy 1 kg more of A.

First, run the old deal at 6500 kg:

In [9]:
mass = 6500
min_cost, optimal_blend, blend_comp = metal_blend(data, mass, bound_array)
print(f'Minimum Cost = {min_cost} dollars')
print(f'Optimal Blend: {optimal_blend}')
print(f'% C: {100*blend_comp[0]}')
print(f'% Cu: {100*blend_comp[1]}')
print(f'% Mn: {100*blend_comp[2]}')

Old_Unit_Cost = min_cost/mass
Old_Total_Cost = min_cost

print(f'total price = {Old_Total_Cost} dollars')
print(f'price per unit = {Old_Unit_Cost} dollars')

Minimum Cost = 8084.428521739293 dollars
Optimal Blend: {'E': 34.60328530629657, 'D': 4.086054622841687e-08, 'C': 80.74099869994772, 'A': 3999.9999998009553, 'G': 2.692619738475979e-07, 'B': 1000.0000002415633, 'F': 1384.6557156411138}
% C: 2.0000000000349347
% Cu: 0.5999999967861396
% Mn: 1.1999999999890791
total price = 8084.428521739293 dollars
price per unit = 1.2437582341137374 dollars


Second, run the new deal at 6500 kg:

In [10]:
data_new = {
    "A": {"C": 2.5, "Cu": 0.0, "Mn": 1.3, "amount": 4001, "cost": 1.20},
    "B": {"C": 3.0, "Cu": 0.0, "Mn": 0.8, "amount": 3000, "cost": 1.50},
    "C": {"C": 0.0, "Cu": 0.3, "Mn": 0.0, "amount": 6000, "cost": 0.90},
    "D": {"C": 0.0, "Cu": 90.0, "Mn": 0.0, "amount": 5000, "cost": 1.30},
    "E": {"C": 0.0, "Cu": 96.0, "Mn": 4.0, "amount": 2000, "cost": 1.45},
    "F": {"C": 0.0, "Cu": 0.4, "Mn": 1.2, "amount": 3000, "cost": 1.20},
    "G": {"C": 0.0, "Cu": 0.6, "Mn": 0.0, "amount": 2500, "cost": 1.00},
}

In [11]:
mass = 6500
min_cost, optimal_blend, blend_comp = metal_blend(data_new, mass, bound_array)
print(f'Minimum Cost = {min_cost} dollars')
print(f'Optimal Blend: {optimal_blend}')
print(f'% C: {100*blend_comp[0]}')
print(f'% Cu: {100*blend_comp[1]}')
print(f'% Mn: {100*blend_comp[2]}')

New_Total_Cost = min_cost
New_Unit_Cost = min_cost/mass

print(f'total cost = {New_Total_Cost} dollars')
print(f'price per unit = {New_Unit_Cost} dollars')

Minimum Cost = 8084.069703435403 dollars
Optimal Blend: {'E': 34.604363017899445, 'D': 4.069410565938277e-08, 'C': 81.10462446945442, 'A': 4000.9999998020844, 'G': 2.7133856606109964e-07, 'B': 999.1666669065416, 'F': 1384.1243454919877}
% C: 2.00000000003459
% Cu: 0.5999999967817308
% Mn: 1.1999999999892195
total cost = 8084.069703435403 dollars
price per unit = 1.2437030312977544 dollars


In [12]:
Cost_Save = Old_Total_Cost - New_Total_Cost
print(f'Saved {Cost_Save} dollars with 1 kg more of A.')

Saved 0.3588183038891657 dollars with 1 kg more of A.


In [13]:
Unit_Cost_Save = Old_Unit_Cost - New_Unit_Cost
Price_Add_A = Unit_Cost_Save + data['A']['cost']
print(f'Price Willing to Pay for Extra A = {Price_Add_A} dollars')

Price Willing to Pay for Extra A = 1.200055202815983 dollars


<!--NAVIGATION-->
< [5.2 Linear Blending Problems](https://jckantor.github.io/cbe30338-2021/05.02-Linear-Blending-Problem.html) | [Contents](toc.html) | [Tag Index](tag_index.html) | [5.4 Gasoline Blending](https://jckantor.github.io/cbe30338-2021/05.04-Gasoline-Blending.html) ><p><a href="https://colab.research.google.com/github/jckantor/cbe30338-2021/blob/master/docs/05.03-Homework_5.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://jckantor.github.io/cbe30338-2021/05.03-Homework_5.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>